### Number of all buildings

In [22]:
from ohsome import OhsomeClient
client = OhsomeClient()

In [50]:
bboxes ="-180.0,-90.0,180.0,90.0" #World
#bboxes ="15.0,50.0,16.0,51.0"
time = "2024-01-01"
fltr = "building=* and geometry:polygon"

response = client.elements.count.post(
    bboxes=bboxes,
    time=time, 
    filter=fltr,
)

df = response.as_dataframe()
display(df)

,value
timestamp,
2024-01-01,587307814.0


### All existing source tags of buildings 

In [24]:
bboxes ="-180.0,-90.0,180.0,90.0" #World
#bboxes ="15.0,50.0,16.0,51.0"
time = "2024-01-01"
fltr = "building=* and geometry:polygon"
groupByKey = "source"

response = client.elements.count.groupByTag.post(
    bboxes=bboxes,
    time=time, 
    filter=fltr,
    groupByKey=groupByKey
)

df = response.as_dataframe()
df.sort_values(by="value", inplace=True, ascending=False)
display(df)

,,value
tag,timestamp,
remainder,2024-01-01,451572489.0
source=microsoft/BuildingFootprints,2024-01-01,20267542.0
source=BAG,2024-01-01,10874708.0
source=cadastre-dgi-fr source : Direction Générale des Impôts - Cadastre. Mise à jour : 2010,2024-01-01,10562478.0
source=cadastre-dgi-fr source : Direction Générale des Impôts - Cadastre. Mise à jour : 2011,2024-01-01,8357089.0
...,...,...
source=MLIT_PLATEAU bldg_767cea36-b9ad-4368-90d1-34d0a1269376,2024-01-01,1.0
source=MLIT_PLATEAU bldg_767e22a5-a240-43a6-9d87-a3ebb1e6de92,2024-01-01,1.0
source=MLIT_PLATEAU bldg_767edcdd-a944-4ada-9de8-2729d6df0281,2024-01-01,1.0


In [27]:
df.to_csv("../data/tags_before_filter.csv")
df.reset_index(inplace=True)

### Filter only buildings with official tags and tags that could signalize AI (if amount of building > 5000)


In [28]:
keywords = [
    # official tags
    "microsoft/BuildingFootprints",
    "esri/Google_Africa_Buildings",
    "esri/Google_Open_Buildings",
    # tags probably identifying building generated by AI
    "AI",
    "ML",
    "fAIr",
    "mapwith",
    # tag possibly identifying building generated by AI - also for searching the typos and combination of other tags
    "footprint",
    "microsoft",
    "google"
    ]


df_filtered = df[
    (df["tag"].str.contains("|".join(keywords)))
    &
    (df["value"] > 5000)
]
display(df_filtered)


,tag,timestamp,value
1,source=microsoft/BuildingFootprints,2024-01-01,20267542.0
42,source=esri_USDOT_Arizona;microsoft/BuildingFo...,2024-01-01,300626.0
146,source=microsoft/BuildingFootprints;esri/USA_N...,2024-01-01,54321.0
205,source=esri_USDOT_Indiana;microsoft/BuildingFo...,2024-01-01,34397.0
212,"source=CUGIR MS footprints, NYS address, ESRI ...",2024-01-01,33697.0
248,source=MML2017,2024-01-01,26647.0
313,source=esri_USDOT_Maryland;microsoft/BuildingF...,2024-01-01,19913.0
319,source=MML,2024-01-01,19693.0
370,source=esri_USDOT_Virgina;microsoft/BuildingFo...,2024-01-01,15621.0
424,source=esri_USDOT_RiverCom911_WA;microsoft/Bui...,2024-01-01,13074.0


### Filter only buildings with typo in official tags

In [36]:
typos = [
    # typos
    "Microsoft/BuildingFootprints",
    "Microsoft/BuildingFootprints.",
    "MicrosoftBuildingFootprints",
    "Microsoft Building Footprints",
    "microsoft/Buildingfootprints",
    "microsoft/building footprint"
    ]


df_filtered2 = df[
    (df["tag"].str.contains("|".join(typos)))
]
display(df_filtered2)

,level_0,index,tag,timestamp,value
263,263,263,source=Microsoft/BuildingFootprints,2024-01-01,25214.0
918,918,918,source=Microsoft/BuildingFootprints.,2024-01-01,3092.0
2060,2060,2060,source=Microsoft Building Footprints,2024-01-01,616.0
5135,5135,5135,source=microsoft/Buildingfootprints,2024-01-01,74.0
99051,99051,99051,source=microsoft/Buildingfootprints;local know...,2024-01-01,1.0
99052,99052,99052,source=microsoft/Buildingfootprints;microsoft/...,2024-01-01,1.0
99054,99054,99054,source=microsoft/building footprint,2024-01-01,1.0
99055,99055,99055,source=microsoft/building footprints,2024-01-01,1.0


### Filter other variants of “source” tag probably identifying building generated by AI

In [49]:
#df.reset_index(inplace=True)
other = [
    # other
    "GlobalMLBuildingFootprints",
    " AI",
    "mapwith",
    "fAIr"
    ]


df_filtered3 = df[
    (df["tag"].str.contains("|".join(other)))
]
display(df_filtered3)

,level_0,index,tag,timestamp,value
2320,2320,2320,source=GlobalMLBuildingFootprints,2024-01-01,475.0
2563,2563,2563,source=fAIr,2024-01-01,379.0
3721,3721,3721,source=AIV Flandres GRB; AIV Flandres imagerie...,2024-01-01,162.0
4928,4928,4928,source=Bing AI,2024-01-01,83.0
6672,6672,6672,source=GRB; AIV aerial imagery,2024-01-01,38.0
6919,6919,6919,source=AIV Flanders GRB; AIV Flanders most rec...,2024-01-01,35.0
9245,9245,9245,source=https://mapwith.ai/,2024-01-01,17.0
11577,11577,11577,source=AIV Flandres GRB; AIV Flandres GRB,2024-01-01,10.0
15350,15350,15350,source=DGAC - SIA / AIP FRANCE,2024-01-01,5.0
15636,15636,15636,source=GlobalMLBuildingFootprints;survey,2024-01-01,5.0
